# Generating PCD and meshes using PDAL and Open3D

In [ ]:
"""
An unusual workflow... Trying to stream line this.

Step 1: load in .las file and write new .las file with only ground points for bare earth mesh (only classification==2)

Step 2: from comandline execute isosurf.json to generate a mesh (***NOTE: Work on getting this into a python env***)


"""

In [14]:
import pdal
import glob
import io
import ipyleaflet
import IPython.display
import ipyvolume.pylab as p3
import matplotlib.cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely.geometry
import scipy.spatial
import pyproj
import open3d as o3d

import os
import requests
import json
import sys
import urllib.request


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [153]:
jr_las = '/home/jose/Desktop/LiDAR_oregon/JR05_2018/points.las'
LC_las = '/home/jose/Desktop/LiDAR_nmsu/SouthEastB1/points.laz'

In [150]:
m = ipyleaflet.Map(center=(45.688698,-121.760025), zoom=13)
m

Map(center=[45.688698, -121.760025], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [160]:
point = shapely.geometry.Point([-121.744288317259716, 45.685326471026336])
#point = shapely.geometry.Point([350338.512888511759229, 3580081.94974302360788])

In [158]:
os.getcwd()

'/home/jose/Github/my_projs/abm_detect/notebooks'

In [161]:
cropper = {
    "pipeline": [ jr_las,
        
        # Cropping out all points that lie 200 units from point.wkt
        {   "type":"filters.crop",
             "a_srs": "EPSG:4326",
             "point":point.wkt,
             "distance": 4000
        },
           
                 # using 12 nn points to determine extremes (aka atmosphere scatter , telephone wires , etc.)
                 # and assigning them class = 7. This will be excluded further down pipeline.
        {
            "type":"filters.outlier",
            "method":"statistical",
            "mean_k":12,
            "multiplier":3.2
        },
               
                 # only commiting classifications 0-6 to memory (avoid class = 7 noise)
        {
            "type": "filters.range",
            "limits":"Classification[1:2]",
        },

                 # using 26 nn points to determine best classification for class = 1(unclassified)
        {
            "type": "filters.neighborclassifier",
            "domain":"Classification[1:1]",
            "k": 26
        },
        
                 
        {
            "type": "filters.range",
            "limits":"Classification[2:2]",
        },
                 
        {
            "type": "writers.las",
            "filename":"for_open3d.las",
            
        },
        
        
    ]}
pipeline = pdal.Pipeline(json.dumps(cropper))
pipeline.validate()
%time n_points = pipeline.execute()
print('Pipeline selected {} points ({:.1f} pts/m2)'.format(n_points, n_points))

CPU times: user 1min 4s, sys: 216 ms, total: 1min 4s
Wall time: 1min 3s
Pipeline selected 2571039 points (2571039.0 pts/m2)


In [56]:
#creating numpy and pandas objects from pdal pipeline
arr = pipeline.arrays[0]
description = arr.dtype.descr
cols = [col for col, __ in description]
df = pd.DataFrame({col: arr[col] for col in cols})
df['X_0'] = df['X']
df['Y_0'] = df['Y']
df['Z_0'] = df['Z']
df['X'] = df['X'] - df['X_0'].min()
df['Y'] = df['Y'] - df['Y_0'].min()
df['Z'] = df['Z'] - df['Z_0'].min()

In [57]:
df#['']

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,Red,Green,Blue,X_0,Y_0,Z_0
0,0.04,97.59,314.81,42335,3,3,0,0,2,-38.0,0,10,2.094057e+08,0,0,0,597783.23,5059950.17,339.26
1,0.19,96.84,314.66,34908,3,3,0,0,2,-38.0,0,10,2.094057e+08,0,0,0,597783.38,5059949.42,339.11
2,0.45,95.53,314.61,30866,3,3,0,1,2,-38.0,0,10,2.094057e+08,0,0,0,597783.64,5059948.11,339.06
3,0.01,97.74,314.71,46114,1,1,0,0,2,-38.0,0,10,2.094057e+08,0,0,0,597783.20,5059950.32,339.16
4,0.31,96.28,314.48,43821,2,2,0,0,2,-38.0,0,10,2.094057e+08,0,0,0,597783.50,5059948.86,338.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571034,575.66,542.19,133.17,46573,1,1,0,0,2,-37.0,0,13,2.094214e+08,0,0,0,598358.85,5060394.77,157.62
2571035,574.86,543.58,132.57,47032,1,1,0,0,2,-37.0,0,13,2.094214e+08,0,0,0,598358.05,5060396.16,157.02
2571036,574.09,544.92,132.01,48474,1,1,0,0,2,-37.0,0,13,2.094214e+08,0,0,0,598357.28,5060397.50,156.46
2571037,573.07,546.72,130.73,46879,1,1,0,0,2,-37.0,0,13,2.094214e+08,0,0,0,598356.26,5060399.30,155.18


# Generating a PCD from XYZ .las data 

In [58]:
# Not necessary to make PCD but nice to have the code to do so
array = np.asarray(df[["X","Y","Z"]])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(array)
o3d.io.write_point_cloud("./TEST.ply", pcd)

True

In [59]:
pcd_load = o3d.io.read_point_cloud("/home/jose/Github/my_projs/abm_detect/notebooks/TEST.ply")
o3d.visualization.draw_geometries([pcd_load])

# Loading in mesh

In [60]:
mesh = o3d.io.read_triangle_mesh("/home/jose/Github/my_projs/abm_detect/notebooks/isosurface_tri.ply",print_progress=True)
o3d.visualization.draw_geometries([mesh])

Reading PLY file: /home/jose/Github/my_projs/abm_detect/notebooks/isosurface_tri.ply[========================================] 100%


In [61]:
print('These are the vertices!!!')
print(np.asarray(mesh.vertices))
print()
print('These are the triangle faces!!!')
print(np.asarray(mesh.triangles))



These are the vertices!!!
[[5.97783e+05 5.05995e+06 3.39260e+02]
 [5.97783e+05 5.05995e+06 3.39110e+02]
 [5.97784e+05 5.05995e+06 3.39060e+02]
 ...
 [5.98357e+05 5.06040e+06 1.56460e+02]
 [5.98356e+05 5.06040e+06 1.55180e+02]
 [5.98355e+05 5.06040e+06 1.54040e+02]]

These are the triangle faces!!!
[[2172766 2172652 1100670]
 [1100670 1727980 2172766]
 [1100670 2172524 1727980]
 ...
 [  29489    3985   29485]
 [  29489   29485    3984]
 [  29484   29485   78585]]


*** NOTE: PDAL allows you to compute normals during pipeline for .las... ###

However...

Not for the commandline pipeline of a isosurface.ply file...
So... untill you can find a way to stream line python + comandline in PDAL. This is the only way to compute normals
of a mesh. intother words... untill you get scipy.spatial.Delaunay() to play well with open3d...

In [62]:
print("Does mesh have normals? (exist: " + str(mesh.has_vertex_normals())+').... :(')


Does mesh have normals? (exist: False).... :(


In [63]:
mesh.compute_vertex_normals() # computing normals

TriangleMesh with 2571039 points and 5139526 triangles.

In [64]:
print("How about now? (normals exist: " + str(mesh.has_vertex_normals())+').... :)')

How about now? (normals exist: True).... :)


In [65]:
o3d.visualization.draw_geometries([mesh])

In [123]:
# PAINTING STUFF!!!
print("Painting the mesh")
mesh.paint_uniform_color([0.55, .15, 0.2])
o3d.visualization.draw_geometries([mesh])


Painting the mesh


# Average Mesh Filtering

In [118]:
# making a copy of original mesh to mess around with it
import copy
mesh1 = copy.deepcopy(mesh)

In [128]:
print('filter with average with 1 iteration')
mesh_out = mesh1.filter_smooth_simple(number_of_iterations=1)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])

filter with average with 1 iteration


In [130]:
print('filter with average with 5 iterations')
mesh_out = mesh1.filter_smooth_simple(number_of_iterations=5)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])



filter with average with 5 iterations


In [126]:
print('filter with average with 5 iterations')
mesh_out = mesh1.filter_smooth_simple(number_of_iterations=10)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])

filter with average with 5 iterations


In [131]:
o3d.visualization.draw_geometries([mesh_out],mesh_show_wireframe=True)

# Laplacian Mesh Filtering

In [133]:
print('filter with Laplacian with 10 iterations')
mesh_out = mesh1.filter_smooth_laplacian(number_of_iterations=10)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])


filter with Laplacian with 10 iterations


In [134]:
o3d.visualization.draw_geometries([mesh_out],mesh_show_wireframe=True)

# Mesh Decimation

In [144]:
mesh_smp = mesh1.simplify_quadric_decimation(target_number_of_triangles=4000000)

In [145]:
o3d.visualization.draw_geometries([mesh_smp],mesh_show_wireframe=True)

In [148]:
print('filter with average with 1 iterations')
mesh_out = mesh_smp.filter_smooth_simple(number_of_iterations=5)
mesh_smp.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out])

filter with average with 1 iterations


In [149]:
o3d.visualization.draw_geometries([mesh_out],mesh_show_wireframe=True)